# 1. Membership Inference Attack (MIA)
对 （Split learning 或 VFL 框架下的）神经网络模型进行membership infernece attack，目前采用了5个攻击方法
* Modified Entropy (Mentropy)
* Entropy
* Confidence
* Correctness
* ML Leaks
类中已经对smashed data进行了softmax

In [1]:
'''
Author: Ruijun Deng
Date: 2024-05-22 22:27:05
LastEditTime: 2024-05-22 22:27:23
LastEditors: Ruijun Deng
FilePath: /PP-Split/examples/membership_inference_attack_and_quantification.ipynb
Description: 
'''
# 导包
import os
import torch
import torchvision
import torch.nn as nn
import numpy as np
import math
import sys
import urllib
import pickle
import argparse

sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')
from ppsplit.attacks.membership_inference.Mentr_attack import MentrAttack # 包含了4种方法的攻击类
from ppsplit.attacks.membership_inference.ML_Leaks_attack import MLLeaksAttack # 包含了ML Leaks攻击类

# 模型
from target_model.models.splitnn_utils import split_weights_client
from target_model.models.ImageClassification.VGG5_9 import VGG,VGG5Decoder,model_cfg
from target_model.models.TableClassification.BankNet import BankNet1,bank_cfg
from target_model.models.TableClassification.CreditNet import CreditNet1,credit_cfg
from target_model.models.TableClassification.PurchaseNet import PurchaseClassifier1,purchase_cfg
# 数据预处理方法
from target_model.data_preprocessing.preprocess_cifar10 import get_cifar10_normalize,get_one_data,deprocess
from target_model.data_preprocessing.preprocess_bank import bank_dataset,preprocess_bank
from target_model.data_preprocessing.preprocess_credit import preprocess_credit
from target_model.data_preprocessing.preprocess_purchase import preprocess_purchase


from target_model.models.splitnn_utils import split_weights_client
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# utils
from ppsplit.utils.utils import create_dir

In [2]:
args = {
        'device':torch.device("cuda:1" if torch.cuda.is_available() else "cpu"),
        # 'device':torch.device("cpu"),
        'dataset':'CIFAR10',
        # 'dataset':'bank',
        # 'dataset':'credit',
        # 'dataset':'purchase',
        # 'result_dir': 'InvMetric-202403',
        'result_dir': 'MIA/',
        'batch_size':32,
        'noise_scale':0, # 防护措施 
        'num_pairs': 200, # RepE
        'topk':10, # smashed data的size
        }
print(args['device'])

cuda:1


In [3]:
# 加载模型和数据集，并从unit模型中切割出client_model
if args['dataset']=='CIFAR10':
    # 超参数
    testset_len = 10000 # 10000个数据一次 整个测试集合的长度
    # split_layer_list = list(range(len(model_cfg['VGG5'])))
    split_layer = 6 # 定成2吧？
    test_num = 3 # 试验序号

    # 关键路径
    # 此时是为了repE
    # unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/VGG5/BN+Tanh/VGG5-params-20ep.pth' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
    unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/VGG5/20240429-RepE/VGG5-params-19ep.pth' # VGG5-BN+Tanh # 存储的是模型参数，不包括模型结构
    results_dir  = f"../results/{args['result_dir']}/VGG5/layer{split_layer}/"
    decoder_route = f"../results/{args['result_dir']}/VGG5/{test_num}/Decoder-layer{split_layer}.pth"

    # 数据集加载
    # trainloader,testloader = get_cifar10_normalize(batch_size = args['batch_size'])
    # one_data_loader = get_one_data(testloader,batch_size = args['batch_size']) #拿到第一个测试数据
    shadow_train_loader, shadow_test_loader = get_cifar10_normalize(batch_size = args['batch_size'])
    target_train_loader, target_test_loader = get_cifar10_normalize(batch_size = args['batch_size'])

    # 切割成client model
    # vgg5_unit.load_state_dict(torch.load(unit_net_route,map_location=torch.device('cpu'))) # 完整的模型
    client_net = VGG('Client','VGG5',split_layer,model_cfg,noise_scale=args['noise_scale'])
    pweights = torch.load(unit_net_route)
    if split_layer < len(model_cfg['VGG5']):
        pweights = split_weights_client(pweights,client_net.state_dict())
    client_net.load_state_dict(pweights)

    class_num = 10

elif args['dataset']=='purchase':
    # 设置一些超参数
    batch_size = 100 # 批大小
    # unit_net_route = '/home/dengruijun/data/FinTech/PP-Split/results/trained_models/Purchase100/Purchase_bestmodel_param.pth' # 待检测模型
    unit_net_route = '/home/dengruijun/data/FinTech/VFL/MIAs/membership_inference_evaluation/adv_reg/training_code/models/purchase_undefended/model_best.pth.tar' # 待检测模型
    split_layer = 8 # 切隔层
    # purchase 数据集 和 模型 导入
    from target_model.models.TableClassification.PurchaseNet import PurchaseClassifier1, purchase_cfg
        
    class PurchaseClassifier(nn.Module):
        def __init__(self,num_classes=100):
            super(PurchaseClassifier, self).__init__()

            self.features = nn.Sequential(
                nn.Linear(600,1024),
                nn.Tanh(),
                nn.Linear(1024,512),
                nn.Tanh(),
                nn.Linear(512,256),
                nn.Tanh(),
                nn.Linear(256,128),
                nn.Tanh(),
            )
            self.classifier = nn.Linear(128,num_classes)
            
        def forward(self,x):
            hidden_out = self.features(x)
            return self.classifier(hidden_out)

    from target_model.data_preprocessing.preprocess_purchase import preprocess_purchase_shadow
    class_num = 100 # Purchase的分类类别数目 # 源论文默认100

    # 模型加载并切割：
    # client_net = PurchaseClassifier1(layer=split_layer)
    # pweights = torch.load(unit_net_route,map_location=device)
    # if split_layer < len(purchase_cfg):
    #     pweights = split_weights_client(pweights,client_net.state_dict())
    # client_net.load_state_dict(pweights)

    client_net = PurchaseClassifier(num_classes=100)
    client_net = torch.nn.DataParallel(client_net).cuda()
    checkpoint = torch.load(unit_net_route)
    client_net.load_state_dict(checkpoint['state_dict'])
    client_net.eval()

    # model = PurchaseClassifier1()
    # model = torch.nn.DataParallel(model).cuda()
    # checkpoint = torch.load(target_model_path)
    # model.load_state_dict(checkpoint['state_dict'])
    # model.eval()

    # 加载数据集
    shadow_train_loader, shadow_test_loader,\
        target_train_loader, target_test_loader = preprocess_purchase_shadow(batch_size=batch_size)
    
else:
    exit(-1)

# 创建文件夹
create_dir(results_dir)

# client_net使用
client_net = client_net.to(args['device'])
client_net.eval()

features.0.weight
features.0.bias
features.1.weight
features.1.bias
features.1.running_mean
features.1.running_var
features.1.num_batches_tracked


VGG(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
  )
  (denses): Sequential()
)

## 1.1 前四种攻击

In [4]:
# 攻击 对象实例化
# ('C', 3, 32, 3, 32*32*32, 32*32*32*3*3*3), # 0
# ('M', 32, 32, 2, 32*16*16, 0),  # 1
# ('C', 32, 64, 3, 64*16*16, 64*16*16*3*3*32), #2
# ('M', 64, 64, 2, 64*8*8, 0), # 3
# ('C', 64, 64, 3, 64*8*8, 64*8*8*3*3*64), # 4
# ('D', 8*8*64, 128, 1, 64, 128*8*8*64), # 5 
# ('D', 128, 10, 1, 10, 128*10)], # 6
MIA = MentrAttack(num_classes = class_num)

# 攻击初始化，推理模型
# 这里 shadow model 和 target model 用的是相同的一个
MIA.prepare_model_performance(client_net, shadow_train_loader, shadow_test_loader, 
                              client_net, target_train_loader, target_test_loader)


(50000, 32, 32, 32)
(10000, 32, 32, 32)
(50000, 32, 32, 32)
(10000, 32, 32, 32)


In [6]:
# 实施MIA攻击
# 设定要执行的攻击类型：默认是4种都执行

benchmark_methods = ['correctness','confidence','entropy','modified entropy'] # 可以取子集
MIA.mem_inf_benchmarks(all_methods=True)


AttributeError: 'MentrAttack' object has no attribute 't_tr_corr'

In [7]:
# 实施MIA攻击，只执行指定的2种攻击
# benchmark_methods = ['correctness','modified entropy'] # 可以取子集
benchmark_methods = ['confidence','entropy','modified entropy'] # 可以取子集
MIA.mem_inf_benchmarks(all_methods=False, benchmark_methods = benchmark_methods)

For membership inference attack via confidence, the attack acc is 514.044
For membership inference attack via entropy, the attack acc is 513.084
For membership inference attack via modified entropy, the attack acc is 514.009


## 1.2 ML Leaks

In [ ]:
# 数据集
from target_model.data_preprocessing.dataset import ListDataset
from target_model.data_preprocessing.preprocess_cifar10 import get_cifar10_normalize_two_train
from torch.utils.data import DataLoader

seen_loader,unseen_loader,test_loader = get_cifar10_normalize_two_train(batch_size=1)

print("seen data length: ",len(seen_loader.dataset))
print("unseen data length: ", len(unseen_loader.dataset))
print("test data length: ", len(test_loader.dataset))

x = iter(seen_loader).next()
print(x[0].shape)
print(x[1].shape)
print(x[1])

In [ ]:
# performance准备 准备attacker的 input feature，处理完存储起来
import pickle
import os

dataset_route = f"../results/{args['result_dir']}/VGG5/layer{split_layer}/"

MIA = MLLeaksAttack(smashed_data_size=args['topk']) # top10

if os.path.isfile(dataset_route+'attack_train_member25000.pkl'):
    print(f"=> loading paired dataset from {dataset_route}")
    with open(dataset_route+'attack_train_member25000.pkl','rb') as f:
        smashed_data_list = pickle.load(file=f)
else:
    print(f"=> making paired dataset...")
    smashed_data_list = MIA.prepare_model_performance(client_net,seen_loader,unseen_loader,test_loader)
    with open(dataset_route+'attack_train_member25000.pkl','wb') as f:
        pickle.dump(obj=smashed_data_list, file=f)

smashed_dataset = ListDataset(smashed_data_list)
# print(len(smashed_dataset))
attack_loader = DataLoader(smashed_dataset, batch_size=100)


In [ ]:
# 查看attack training数据集：
all_labels = [d[1] for d in smashed_data_list]
all_features = [d[0] for d in smashed_data_list]
print("第一个smashed data形状:",all_features[0].shape)
print("第一个smashed data取值:",all_features[0])
print("所有标签: ",all_labels)

# 第一个数据的shape：
print("第一个数据的shape: ")
(x,y) = iter(attack_loader).next()
print(x.shape)
print(y.shape)

In [ ]:
# 训练attack model
attack_loader = DataLoader(smashed_dataset, batch_size=32)

MIA.train_attack_model(attack_loader,optimizer=None,epochs=20)

In [ ]:
# 使用 attack model 进行 MIA攻击
data_loader = DataLoader(smashed_dataset, batch_size=1)

MIA.MIA_test(data_loader)

In [ ]:
# 测试 F
import torch.nn.functional as F
x=torch.tensor([[1,1,1,],[4,4,4]],dtype=float)
print(F.softmax(x,dim=1))

In [ ]:
x = torch.tensor([1])
y = x.squeeze()
print(x)
print([i for i in y.tolist()])

# 2. Quantification for MIA
用 privacy score 量化 MIA攻击隐私情况

In [ ]:
from ppsplit.quantification.privacy_risk_score.privacy_risk_score import calculate_risk_score # 导包
# private risk score calculation
risk_score = calculate_risk_score(MIA.s_tr_m_entr, MIA.s_te_m_entr,  # shadow model mentropy
                                MIA.s_tr_labels, MIA.s_te_labels,  # shadow model labels
                                MIA.t_tr_m_entr, MIA.t_tr_labels) # target model mentropy & labels
print(f"target model 的 训练数据 (共{len(risk_score)}）的MIA隐私泄漏 privacy score 为：", risk_score)
print(f"平均privacy score为{risk_score.mean()}")